In [1]:
!pip install -U datasets

In [2]:
# %% 1. การติดตั้งและ Import Library

# ติดตั้ง library ที่จำเป็น
!pip install -q transformers torch accelerate

import torch
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
from tqdm.auto import tqdm

# ตรวจสอบและตั้งค่าอุปกรณ์ (GPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"อุปกรณ์ที่ใช้: {DEVICE}")


อุปกรณ์ที่ใช้: cuda


In [3]:
# %% 2. การเตรียมข้อมูล

print("กำลังโหลด ThaiSum Dataset...")
from torch.utils.data import IterableDataset
    # โหลดชุดข้อมูลข่าวจาก thaisum
dataset = load_dataset("pythainlp/thaisum", "default", split='train', streaming=True)
data_list = []
for example in dataset:
    # Filter out examples where 'body' or 'summary' is None
    if example['body'] is not None and example['summary'] is not None:
        data_list.append({
            'body': example['body'],
            'summary': example['summary'],
            'tags': example['tags']
        })
    # Optional: Limit the number of examples for faster testing
    if len(data_list) >= 1000: # Collect slightly more than needed for sampling
        break

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data_list)

print(f"โหลดข้อมูลดิบสำเร็จ จำนวน {len(df)} รายการ")
# --- สร้างข้อมูลสำหรับทดสอบ (test.csv) ---
# เราไม่ต้องการ train.csv สำหรับวิธีนี้ เพราะเป็นการทำ few-shot prompting
test_df = df[['body']].copy().dropna().sample(n=500, random_state=42)
test_df.rename(columns={'body': 'document_text'}, inplace=True)
test_df['document_id'] = test_df.index.astype(int)
test_df.reset_index(drop=True, inplace=True)
print(f"\nเตรียมข้อมูล test_df สำเร็จ จำนวน {len(test_df)} ตัวอย่าง")
print("ตัวอย่างข้อมูล Test:")
print(test_df.head())



กำลังโหลด ThaiSum Dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


โหลดข้อมูลดิบสำเร็จ จำนวน 1000 รายการ

เตรียมข้อมูล test_df สำเร็จ จำนวน 500 ตัวอย่าง
ตัวอย่างข้อมูล Test:
                                       document_text  document_id
0  เมื่อ 9 ธ.ค. สำนักข่าวต่างประเทศรายงานอ้างโฆษก...          521
1  ตำรวจ สน.โคกครามสรุปสำนวนคดี ไอ้ต้อม-ไอ้เอ็กซ์...          737
2  ส้ม-ธัญสินี พรมสุทธิ์, ขึ้นสเตตัสบนเฟซบุ๊กส่วน...          740
3  เมื่อเวลา 11.00 น. วันที่ 18 ธ.ค.58 กรมอุตุนิย...          660
4  ผู้สื่อข่าวจังหวัดอุบลราชธานี รายงานว่า เมื่อช...          411


In [4]:
# %% 3. การโหลดโมเดล Qwen2 และ Tokenizer

model_id = "Qwen/Qwen2-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [5]:
# %% 4. การออกแบบ Prompt สำหรับงาน Summarization และ Keyword Extraction

# สร้างโครงสร้าง Prompt แบบ Chat ที่ชัดเจนและมีประสิทธิภาพ
# 1. System Prompt: กำหนดบทบาท, หน้าที่, และ "กฎเหล็ก" คือต้องตอบเป็น JSON เท่านั้น
# 2. Few-Shot Examples: ตัวอย่างคำถาม-คำตอบที่ดี 2 ชุด
# 3. Final User Prompt: โครงคำถามสุดท้ายสำหรับใส่เอกสารจริง

chat_template = [
    {
        "role": "system",
        "content": "คุณคือผู้เชี่ยวชาญด้านการวิเคราะห์เอกสารนโยบายภาครัฐ หน้าที่ของคุณคือ 1. สรุปใจความสำคัญของเอกสาร และ 2. สกัดคีย์เวิร์ดหลักออกมา คุณต้องตอบกลับเป็นโครงสร้าง JSON ที่กำหนดเท่านั้น ห้ามมีข้อความอธิบายอื่นใดๆ นอกเหนือจาก JSON Object"
    },
    {
        "role": "user",
        "content": """สำหรับเอกสารต่อไปนี้: "คณะรัฐมนตรีมีมติอนุมัติหลักการร่างพระราชกฤษฎีกากำหนดเขตที่ดินที่จะเวนคืนในท้องที่ตำบลบางพูด และตำบลบ้านใหม่ อำเภอปากเกร็ด จังหวัดนนทบุรี เพื่อสร้างและขยายทางหลวงท้องถิ่นสายเชื่อมระหว่างถนนติวานนท์กับถนนแจ้งวัฒนะ ตามที่กระทรวงมหาดไทยเสนอ เพื่อแก้ไขปัญหาจราจรและสนับสนุนการพัฒนาพื้นที่โดยรอบ"

1. จงสรุปสาระสำคัญของเอกสารนี้ให้เหลือไม่เกิน 3 ประโยค
2. จงสกัดคีย์เวิร์ดที่สำคัญที่สุด
จงตอบในรูปแบบ JSON เท่านั้น"""
    },
    {
        "role": "assistant",
        "content": """{"summary": "คณะรัฐมนตรีอนุมัติหลักการร่าง พ.ร.ฎ. เวนคืนที่ดินใน อ.ปากเกร็ด จ.นนทบุรี เพื่อสร้างและขยายถนนเชื่อมระหว่างถนนติวานนท์และถนนแจ้งวัฒนะ การดำเนินการนี้มีวัตถุประสงค์เพื่อแก้ไขปัญหาการจราจรในพื้นที่ดังกล่าว", "keywords": ["เวนคืนที่ดิน", "นนทบุรี", "สร้างถนน", "แก้ปัญหาจราจร", "คณะรัฐมนตรี"]}"""
    },
    {
        "role": "user",
        "content": """สำหรับเอกสารต่อไปนี้: "กรมควบคุมโรค กระทรวงสาธารณสุข เปิดเผยสถานการณ์การแพร่ระบาดของโรคไข้เลือดออกประจำสัปดาห์ พบผู้ป่วยเพิ่มขึ้นในหลายจังหวัด โดยเฉพาะในเขตเมืองและชุมชนแออัด จึงขอความร่วมมือประชาชนให้ช่วยกันกำจัดแหล่งเพาะพันธุ์ยุงลายในบ้านและบริเวณรอบบ้านอย่างสม่ำเสมอ เพื่อตัดวงจรการระบาด"

1. จงสรุปสาระสำคัญของเอกสารนี้ให้เหลือไม่เกิน 3 ประโยค
2. จงสกัดคีย์เวิร์ดที่สำคัญที่สุด
จงตอบในรูปแบบ JSON เท่านั้น"""
    },
    {
        "role": "assistant",
        "content": """{"summary": "กรมควบคุมโรครายงานสถานการณ์ไข้เลือดออกพบผู้ป่วยเพิ่มขึ้น โดยเฉพาะในเขตเมือง จึงขอความร่วมมือประชาชนช่วยกำจัดแหล่งเพาะพันธุ์ยุงลาย เพื่อป้องกันการระบาด", "keywords": ["ไข้เลือดออก", "กรมควบคุมโรค", "กำจัดยุงลาย", "สถานการณ์ระบาด", "สาธารณสุข"]}"""
    },
    {
        "role": "user",
        "content": """สำหรับเอกสารต่อไปนี้: "{document_text}"

1. จงสรุปสาระสำคัญของเอกสารนี้ให้เหลือไม่เกิน 3 ประโยค
2. จงสกัดคีย์เวิร์ดที่สำคัญที่สุด
จงตอบในรูปแบบ JSON เท่านั้น"""
    }
]

# ทดลองสร้าง prompt ที่สมบูรณ์สำหรับเอกสารตัวอย่าง
sample_doc = test_df['document_text'].iloc[0]
final_prompt_example = chat_template[-1]["content"].format(document_text=sample_doc)
print("--- ตัวอย่าง Prompt ส่วนสุดท้ายที่จะส่งให้โมเดล ---")
print(final_prompt_example)


--- ตัวอย่าง Prompt ส่วนสุดท้ายที่จะส่งให้โมเดล ---
สำหรับเอกสารต่อไปนี้: "เมื่อ 9 ธ.ค. สำนักข่าวต่างประเทศรายงานอ้างโฆษกประจำทำเนียบขาวว่า รัฐบาลสหรัฐฯ ได้มีคำสั่งยกระดับมาตรการด้านความปลอดภัยตามสถานเอกอัครราชทูตของสหรัฐฯ ทั่วโลก รวมทั้งหน่วยงานต่างๆ ของสหรัฐฯ ในต่างแดน ขณะที่นาวิกโยธินอเมริกันซึ่งประจำการอยู่ในหลายประเทศได้รับคำสั่งให้อยู่ในความเตรียมพร้อม เนื่องจากมีความเสี่ยงจะเกิดเหตุรุนแรงมากขึ้น ขณะจะมีการเปิดเผยรายงานสรุปของคณะกรรมการด้านข่าวกรองของวุฒิสภาสหรัฐฯ เกี่ยวกับวิธีการสอบปากคำของสำนักงานข่าวกรองกลาง (CIA) ที่ปฏิบัติต่อบรรดาผู้ต้องสงสัยเป็นสมาชิกกลุ่มก่อการร้าย ความยาว 480 หน้า ในวันอังคารที่ 9 ธ.ค. นี้ (ตามเวลาท้องถิ่น),สำหรับรายละเอียดของรายงานสรุปของคณะกรรมการด้านข่าวกรองของวุฒิสภาดังกล่าว คาดว่าจะเป็นรายละเอียดเกี่ยวกับวิธีการสอบปากคำของเจ้าหน้าที่ซีไอเอ สมัยรัฐบาลอดีตประธานาธิบดีจอร์จ ดับเบิลยู.บุช ที่ได้กระทำต่อสมาชิกกลุ่มก่อการร้ายอัลเคดา หลังเกิดเหตุการณ์วินาศกรรมช็อกโลก 9/11 ซึ่งอาจรวมถึงวิธีการทรมาน กระทำทารุณ ที่ซีไอเอใช้กับผู้ต้องสงสัยระดับแกนนำในการรีดข้อม

In [6]:
# %% 5. การเริ่ม Pipeline การทำนายผล

results = []
# เพื่อความรวดเร็วในการทดลอง เราจะทำแค่ 20 เอกสารแรก
# ในการแข่งจริง ให้ใช้ test_df ทั้งหมด
test_subset = test_df.head(20)

print(f"กำลังเริ่มทำนายผลสำหรับเอกสาร {len(test_subset)} ฉบับ...")

for index, row in tqdm(test_subset.iterrows(), total=len(test_subset)):
    document_id = row['document_id']
    document_text = row['document_text']

    # สร้าง Prompt สำหรับเอกสารปัจจุบัน
    current_chat = [
        *chat_template[:-1],
        {"role": "user", "content": chat_template[-1]["content"].format(document_text=document_text)}
    ]

    # แปลง prompt เป็น token id ตามรูปแบบของโมเดล
    # เราจะตัดข้อความที่ยาวเกิน 2048 token เพื่อป้องกันปัญหาหน่วยความจำ
    inputs = tokenizer.apply_chat_template(
        current_chat,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(DEVICE)

    # จำกัดความยาวของ input เพื่อไม่ให้เกินขีดจำกัดของโมเดล
    if inputs.shape[1] > 2048:
        inputs = inputs[:, -2048:]


    # สั่งให้โมเดลสร้างข้อความ
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=512,  # เพิ่ม max_tokens สำหรับงานสรุปความ
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    response_text = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)

    # --- ส่วนของการ Parse ผลลัพธ์ JSON ที่รัดกุม ---
    try:
        start_index = response_text.find('{')
        end_index = response_text.rfind('}')
        if start_index != -1 and end_index != -1:
            json_part = response_text[start_index : end_index + 1]
            parsed_output = json.loads(json_part)

            summary = parsed_output.get('summary', '')
            keywords_list = parsed_output.get('keywords', [])

            results.append({
                'document_id': document_id,
                'summary': summary,
                'keywords': ", ".join(keywords_list)  # แปลง list เป็น string
            })
    except (json.JSONDecodeError, AttributeError) as e:
        print(f"\n[Error] ไม่สามารถ Parse JSON สำหรับ document_id {document_id}.")
        print(f"  -> ผลลัพธ์จากโมเดล: '{response_text}'")

print("\nการทำนายผลเสร็จสิ้น")


กำลังเริ่มทำนายผลสำหรับเอกสาร 20 ฉบับ...


  0%|          | 0/20 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The followi


[Error] ไม่สามารถ Parse JSON สำหรับ document_id 280.
  -> ผลลัพธ์จากโมเดล: '{"summary": "เอกสารนี้เป็นรายการมวยที่ช่อง7 รายงานวันที่ 14 เม.ย. ประกอบด้วยการวิเคราะห์ผลมวยจากหลายเวที รวมถึงการถ่ายทอดสดมวยไทย 7 สี ช่อง 7 และการวิเคราะห์ผลมวยจากศึกมวยดีวิถีไทย ช่อง 28 เอสดี และศึกบางระจัน เวทีราชดำเนิน"}, 
"keywords": ["ช่อง7", "มวยไทย", "มวยดีวิถีไทย", "เวทีราชดำเนิน", "มวยไทย 7 สี", "มวย", "วิเคราะห์ผลมวย"]}'


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



การทำนายผลเสร็จสิ้น


In [7]:
# %% 6. การสร้างไฟล์ Submission

print("กำลังสร้างไฟล์ submission.csv...")

if not results:
    print("ไม่พบผลลัพธ์ที่ถูกต้อง! ไม่สามารถสร้างไฟล์ submission ได้")
else:
    submission_df = pd.DataFrame(results)
    submission_df = submission_df[['document_id', 'summary', 'keywords']]

    print("\nตัวอย่างผลลัพธ์ในไฟล์ Submission:")
    print(submission_df.head())

    submission_df.to_csv("submission.csv", index=False)
    print("\nสร้างไฟล์ submission.csv สำเร็จ!")


กำลังสร้างไฟล์ submission.csv...

ตัวอย่างผลลัพธ์ในไฟล์ Submission:
   document_id                                            summary  \
0          521  รัฐบาลสหรัฐฯ ยกระดับมาตรการความปลอดภัยตามสถานเ...   
1          737  ตำรวจได้จับกุมผู้ต้องหาที่ก่อเหตุฆ่าชิงทรัพย์ข...   
2          740  ส้ม-ธัญสินี พรมสุทธิ์ ประกาศโสดหลังจากคบกับจิน...   
3          660  กรมอุตุนิยมวิทยารายงานสภาพอากาศหนาวเย็นในภาคเห...   
4          411  พายุฤดูร้อนพัดถล่มหมู่บ้านหัวเรือ อ.เมืองอุบลร...   

                                            keywords  
0  รัฐบาลสหรัฐฯ, มาตรการความปลอดภัย, สถานเอกอัครร...  
1                                                     
2  ส้ม-ธัญสินี พรมสุทธิ์, จิน-นครินทร์ธารา, โสด, ...  
3  กรมอุตุนิยมวิทยา, อากาศหนาวเย็น, คลื่นลมแรง, ภ...  
4  พายุฤดูร้อน, อุบลราชธานี, ความเสียหายบ้าน, ชาว...  

สร้างไฟล์ submission.csv สำเร็จ!


In [8]:
results

[{'document_id': 521,
  'summary': 'รัฐบาลสหรัฐฯ ยกระดับมาตรการความปลอดภัยตามสถานเอกอัครราชทูตทั่วโลก เนื่องจากมีความเสี่ยงการเกิดเหตุรุนแรงมากขึ้น ขณะที่คณะกรรมการด้านข่าวกรองของวุฒิสภาสหรัฐฯ จะเปิดเผยรายงานสรุปเกี่ยวกับวิธีการสอบปากคำของสำนักงานข่าวกรองกลาง (CIA) ที่ปฏิบัติต่อบรรดาผู้ต้องสงสัยเป็นสมาชิกกลุ่มก่อการร้าย ขณะที่รัฐบาลแคนาดาได้มีคำสั่งปิดสถานเอกอัครราชทูตแคนาดาประจำกรุงไคโร เนื่องจากกังวลเกี่ยวกับความปลอดภัย',
  'keywords': 'รัฐบาลสหรัฐฯ, มาตรการความปลอดภัย, สถานเอกอัครราชทูต, คณะกรรมการด้านข่าวกรองของวุฒิสภาสหรัฐฯ, รายงานสรุป, วิธีการสอบปากคำของสำนักงานข่าวกรองกลาง (CIA), รัฐบาลแคนาดา, สถานเอกอัครราชทูตแคนาดา, กรุงไคโร, ความปลอดภัย'},
 {'document_id': 737,
  'summary': 'ตำรวจได้จับกุมผู้ต้องหาที่ก่อเหตุฆ่าชิงทรัพย์ของนักศึกษาที่มหาวิทยาลัยศรีนครินทรวิโรฒ ซึ่งผู้ต้องหาเคยติดคุกมา 8 ครั้งแล้ว และได้ส่งสำนวนคดีให้อัยการพิเศษเพื่อพิจารณาสั่งฟ้อง. นักศึกษาที่ถูกฆ่าชิงทรัพย์ได้รับการฌาปนกิจศพที่วัดกลางคลองสาม อ.คลองหลวง จ.ปทุมธานี โดยมีการจัดงานศพและมีการแสดงจุดยืนต่อผู้ต้องหา

In [9]:
submission_df

,document_id,summary,keywords
0,521,รัฐบาลสหรัฐฯ ยกระดับมาตรการความปลอดภัยตามสถานเ...,"รัฐบาลสหรัฐฯ, มาตรการความปลอดภัย, สถานเอกอัครร..."
1,737,ตำรวจได้จับกุมผู้ต้องหาที่ก่อเหตุฆ่าชิงทรัพย์ข...,
2,740,ส้ม-ธัญสินี พรมสุทธิ์ ประกาศโสดหลังจากคบกับจิน...,"ส้ม-ธัญสินี พรมสุทธิ์, จิน-นครินทร์ธารา, โสด, ..."
3,660,กรมอุตุนิยมวิทยารายงานสภาพอากาศหนาวเย็นในภาคเห...,"กรมอุตุนิยมวิทยา, อากาศหนาวเย็น, คลื่นลมแรง, ภ..."
4,411,พายุฤดูร้อนพัดถล่มหมู่บ้านหัวเรือ อ.เมืองอุบลร...,"พายุฤดูร้อน, อุบลราชธานี, ความเสียหายบ้าน, ชาว..."
5,678,เศรษฐกิจไทยคาดว่าจะฟื้นตัวขึ้นเล็กน้อยในปี 256...,"เศรษฐกิจไทย, ฟื้นตัว, ปี 2563, การส่งออก, การท..."
6,626,พ.ต.ท.วิชัย จำปาทุม ผบ.ร้อย ตชด.326 สั่งการให้...,"นายทวีศักดิ์ อุณหพัฒนา, พ.ต.ท.วิชัย จำปาทุม, ก..."
7,513,ออสเตรเลียเผชิญกับอุทกภัยครั้งใหญ่ ทำให้มีประช...,"อุทกภัย, ออสเตรเลีย, หมอกลงจัด, จีน, ประชาชนติ..."
8,859,ตำรวจลงพื้นที่ตรวจสอบชุมชนประชาชื่น ช่วงเทศกาล...,"ตำรวจ, ชุมชนประชาชื่น, เทศกาลสงกรานต์, มาตรการ..."
9,136,เกิดเหตุยิงกันในหมู่บ้านกานดา ถนนพระราม2 ช่วง ...,"เหตุยิง, ผู้เสียชีวิต, วัยรุ่น, หมู่บ้านกานดา,..."
